Importation des données

In [1]:
import pandas as pd
import json


In [3]:
path = "C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\PIP\\DonneesBrutes\\Voiture\\voitures-par-commune-par-energie.json"
with open(path, 'r', encoding='utf-8') as f:
    voiture = json.load(f)
# Rajoute un separateur ; pour les colonnes
code_postal1 = pd.read_csv("C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\PIP\\DonneesBrutes\\Voiture\\code_postal.csv", sep=',')


In [ ]:
# Extraire les colonnes "code_departement" et "nom_departement" de 'code_postal1' et supprimer les doublons
departement = code_postal1[["code_departement", "nom_departement"]].drop_duplicates()

# Convertir la colonne "code_departement" en type chaîne de caractères (str)
departement["code_departement"] = departement["code_departement"].astype(str)

# Réassigner la colonne "code_departement" pour s'assurer que le type est bien conservé (cette ligne est redondante)
departement["code_departement"] = departement["code_departement"]

# Extraire les colonnes "code_departement", "code_region", et "nom_region" de 'code_postal1' et supprimer les doublons
region = code_postal1[["code_departement", "code_region", "nom_region"]].drop_duplicates()

# Convertir la colonne "code_departement" en type chaîne de caractères (str)
region["code_departement"] = region["code_departement"].astype(str)

# Appliquer un remplissage avec des zéros pour s'assurer que les codes départementaux ont deux chiffres
region["code_departement"] = region["code_departement"].apply(lambda x: x.zfill(2))

# Créer un DataFrame 'df' à partir des données 'voiture'
df = pd.DataFrame(voiture)

# Exclure les lignes du DataFrame 'df' où la colonne 'codgeo' commence par '97' (correspondant aux DOM-TOM)
df = df[~df['codgeo'].str.startswith('97')]

# Définir une fonction pour extraire le code département à partir du code géographique 'codgeo'
def get_code_dep(codgeo):
    if codgeo.startswith('97'):  # Si le code commence par '97', extraire les trois premiers caractères (DOM-TOM)
        return codgeo[:3]
    return codgeo[:2]  # Sinon, extraire les deux premiers caractères (métropole)

# La fonction 'get_code_dep' pourra être utilisée pour créer une colonne 'code_dep' dans un DataFrame


In [54]:
# Extraire les colonnes "code_departement" et "nom_departement" de 'code_postal1' et supprimer les doublons
departement = code_postal1[["code_departement", "nom_departement"]].drop_duplicates()

# Convertir la colonne "code_departement" en type chaîne de caractères (str)
departement["code_departement"] = departement["code_departement"].astype(str)

# Remplir les codes départementaux avec des zéros pour s'assurer qu'ils ont deux chiffres (ex. '1' -> '01')
departement["code_departement"] = departement["code_departement"].apply(lambda x: x.zfill(2))

# Joindre le DataFrame 'departement' avec 'df' sur "code_dep" et "code_departement", en utilisant une jointure à gauche
df2 = df.merge(departement, left_on="code_dep", right_on="code_departement", how="left")

# Supprimer la colonne redondante "code_departement" après la jointure
df2.drop(columns=["code_departement"], inplace=True)

# Extraire les colonnes "code_departement", "code_region", et "nom_region" de 'code_postal1' et supprimer les doublons
region = code_postal1[["code_departement", "code_region", "nom_region"]].drop_duplicates()

# Convertir la colonne "code_departement" en type chaîne de caractères (str) et la remplir avec des zéros
region["code_departement"] = region["code_departement"].astype(str)
region["code_departement"] = region["code_departement"].apply(lambda x: x.zfill(2))

# Obtenir une liste unique des codes départements dans 'df2' et dans 'region'
cp_ve_code_geo = df2["code_dep"].unique().tolist()
cp_code_geo = region["code_departement"].unique().tolist()

# Identifier les codes présents dans 'df2' mais absents dans 'region'
cp_not_in_code_geo = [i for i in cp_ve_code_geo if i not in cp_code_geo]

# Joindre 'region' avec 'df2' sur "code_dep" et "code_departement", en utilisant une jointure à gauche
df2 = df2.merge(region, left_on="code_dep", right_on="code_departement", how="left")

# Supprimer la colonne redondante "code_departement" après la jointure
df2.drop(columns=["code_departement"], inplace=True)

# Convertir la colonne "code_region" en type chaîne de caractères (str)
df2["code_region"] = df2["code_region"].astype(str)

# Charger un fichier CSV contenant des informations sur les voitures par commune et département
nb_voiture_commune_dep = pd.read_csv(
    "C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\PIP\\DonneesBrutes\\Voiture\\nb_voiture_commune_dep.csv"
)

# Convertir la colonne "codgeo" en type chaîne de caractères
nb_voiture_commune_dep["codgeo"] = nb_voiture_commune_dep["codgeo"].astype(str)

# Trier le DataFrame par "codgeo" et "date_arrete"
nb_voiture_commune_dep = nb_voiture_commune_dep.sort_values(by=["codgeo", "date_arrete"])

# Filtrer les données pour un code géographique spécifique (exemple : '04151')
nb_voiture_commune_dep[nb_voiture_commune_dep["codgeo"] == "04151"]

# Ajouter une colonne "annee" correspondant à l'année extraite de "date_arrete"
nb_voiture_commune_dep["annee"] = pd.to_datetime(nb_voiture_commune_dep["date_arrete"]).dt.year

# Créer une copie du DataFrame pour manipuler les données par année
nb_voiture_commune_dep_anne = nb_voiture_commune_dep.copy()

# Supprimer la colonne "date_arrete" de la copie
nb_voiture_commune_dep_anne.drop(columns=["date_arrete"], inplace=True)

# Supprimer les doublons dans 'nb_voiture_commune_dep_anne' en gardant uniquement la dernière occurrence pour chaque "codgeo" et "annee"
nb_voiture_commune_dep_anne2 = nb_voiture_commune_dep_anne.drop_duplicates(subset=["codgeo", "annee"], keep="last")

# Supprimer les colonnes "epci" et "libepci" du DataFrame nettoyé
nb_voiture_commune_dep_anne2.drop(columns=["epci", "libepci"], inplace=True)

# Charger un fichier CSV contenant les informations sur le nombre de voitures par année au niveau des départements
nb_voiture_commune_dep = pd.read_csv(
    "C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\PIP\\DonneesBrutes\\Voiture\\nb_voiture_annee_cdr.csv"
)

# Charger un autre fichier CSV contenant des prédictions régionales
nb_voiture_dep = pd.read_csv(
    "C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\Dossier_git\\data\\Pred_Reg_tout.csv",
    delimiter=";",
    encoding="utf-8"
)

# Charger un fichier CSV contenant les données départementales sur les voitures
nb_voiture_dep = pd.read_csv(
    "C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\Dossier_git\\data\\nb_voiture_dep.csv",
    encoding="utf-8"
)

# Remplir les codes départementaux avec des zéros pour s'assurer qu'ils ont deux chiffres
nb_voiture_dep["code_dep"] = nb_voiture_dep["code_dep"].apply(lambda x: x.zfill(2))


In [36]:
import pandas as pd

# Charger les données depuis le fichier CSV "Pred_Reg_tout.csv" avec un séparateur point-virgule et un encodage UTF-8
bornes_completes = pd.read_csv(
    "C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\Dossier_git\\data\\Pred_Reg_tout.csv",
    delimiter=";",
    encoding="utf-8"
)

# Sélectionner les colonnes pertinentes dans le DataFrame `nb_voiture_commune_dep`
nb_voiture_commune_dep = nb_voiture_commune_dep[[
    "code_dep", "nom_departement", "nb_vp_rechargeables_el", "nb_vp"
]]

# Grouper les données par 'nom_departement' et 'code_dep' pour calculer les sommes des colonnes "nb_vp_rechargeables_el" et "nb_vp"
nb_voiture_commune_dep = nb_voiture_commune_dep.groupby(['nom_departement', 'code_dep']).agg({
    'nb_vp_rechargeables_el': 'sum',
    'nb_vp': 'sum'
}).reset_index()

# Convertir la colonne "code_dep" en type chaîne de caractères
nb_voiture_commune_dep["code_dep"] = nb_voiture_commune_dep["code_dep"].astype(str)

# Ajouter un zéro devant les codes départementaux à un chiffre pour garantir que tous les codes ont deux chiffres
nb_voiture_commune_dep["code_dep"] = nb_voiture_commune_dep["code_dep"].apply(lambda x: x.zfill(2))

# Charger un autre fichier CSV contenant des informations sur les voitures par département
nb_voiture_commune_dep = pd.read_csv(
    "C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\PIP\\DonneesBrutes\\Voiture\\nb_voiture_commune_dep.csv"
)

# Sélectionner uniquement les colonnes nécessaires pour effectuer des agrégations
nb_voiture_commune_dep_total = nb_voiture_commune_dep[[
    "code_dep", "nom_departement", "nb_vp_rechargeables_el", "nb_vp"
]]

# Grouper à nouveau les données par 'nom_departement' et 'code_dep' pour calculer les totaux des colonnes numériques
nb_voiture_commune_dep_total = nb_voiture_commune_dep_total.groupby(['nom_departement', 'code_dep']).agg({
    'nb_vp_rechargeables_el': 'sum',
    'nb_vp': 'sum'
}).reset_index()


# Affichage année et granularité

In [ ]:
nb_voiture_commune_dep2 = pd.read_csv("C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\PIP\\DonneesBrutes\\Voiture\\nb_voiture_annee_cdr.csv")
bornes = pd.read_csv("C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\Dossier_git\\data\\Bornes_nettoye3.csv")
bornes_brut = pd.read_csv("C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\Dossier_git\\data\\Bornes_nettoye2.csv", delimiter = ";")


# fait une jointure entre bornes et bornes_brut avec les index
bornes = bornes.merge(bornes_brut, left_index=True, right_index=True)

In [46]:
import pandas as pd

# Chargement des données
bornes = pd.read_csv("C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\Dossier_git\\data\\Bornes_completes.csv")
nb_voiture_commune_dep2 = pd.read_csv("C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\PIP\\DonneesBrutes\\Voiture\\nb_voiture_commune_dep.csv")

# Ajout des colonnes nécessaires
bornes["nb_borne"] = 1
bornes = bornes[['nom_amenageur', 'nom_operateur', "commune", "code_insee", 
                 "Departement_selon_insee", "Annee", "nom_departement", 
                 "code_region", "nom_region", "nb_borne"]]

# Séparation des bornes avec ou sans date
bornes_sans_date = bornes[bornes["Annee"].isna()]
bornes_avec_date = bornes[~bornes["Annee"].isna()]

# Agrégation par différents niveaux
bornes_avec_date = bornes_avec_date.groupby(['nom_amenageur', 'nom_operateur', "commune", 
                                             "code_insee", "Departement_selon_insee", 
                                             "Annee", "nom_departement", "code_region", 
                                             "nom_region"]).agg({'nb_borne': 'sum'}).reset_index()

bornes_avec_date_commune = bornes_avec_date.groupby(['nom_amenageur', 'nom_operateur', "commune", 
                                                     "code_insee", "nom_departement", 
                                                     "Departement_selon_insee", "nom_region", 
                                                     "code_region", "Annee"]).agg({'nb_borne': 'sum'}).reset_index()

bornes_avec_date_commune['Annee'] = bornes_avec_date_commune['Annee'].astype(int)
bornes_avec_date_commune = bornes_avec_date_commune.sort_values(by=['code_insee', 'Annee'])

# Création de la plage complète des années pour chaque couple
annee_min, annee_max = bornes_avec_date_commune['Annee'].min(), bornes_avec_date_commune['Annee'].max()
colonnes_couple = ['nom_amenageur', 'nom_operateur', 'commune', 'code_insee', 
                   'nom_departement', 'Departement_selon_insee', 'nom_region', 'code_region']

couples_uniques = bornes_avec_date_commune[colonnes_couple].drop_duplicates()
plage_complete = [
    {**row.to_dict(), 'Annee': annee} 
    for _, row in couples_uniques.iterrows() 
    for annee in range(annee_min, annee_max + 1)
]
plage_complete_df = pd.DataFrame(plage_complete)

# Fusion et remplissage
bornes_completes = pd.merge(
    plage_complete_df,
    bornes_avec_date_commune,
    on=['nom_amenageur', 'nom_operateur', 'commune', 'code_insee', 
        'nom_departement', 'Departement_selon_insee', 'nom_region', 
        'code_region', 'Annee'],
    how='left'
)
bornes_completes['nb_borne'] = bornes_completes['nb_borne'].fillna(0)

# Calcul des sommes cumulées
bornes_completes['nb_borne_cumul'] = bornes_completes.sort_values(by=['code_insee', 'commune', 'Annee']) \
    .groupby(['commune', 'code_insee'])['nb_borne'].cumsum()

# Export des données groupées
bornes_com = bornes_completes[["commune", "code_insee", "Annee", "nb_borne_cumul"]]
bornes_dep = bornes_completes.groupby(["Departement_selon_insee", "nom_departement", "Annee"]).agg({'nb_borne_cumul': 'sum'}).reset_index()
bornes_reg = bornes_completes.groupby(["code_region", "nom_region", "Annee"]).agg({'nb_borne_cumul': 'sum'}).reset_index()

# Groupement des données de voitures électriques par région et département
nb_voiture_reg = nb_voiture_commune_dep2.groupby(['nom_region', 'code_region', 'reg_code_name', "annee"]).agg({
    'nb_vp_rechargeables_el': 'sum',
    'nb_vp': 'sum'
}).reset_index()

nb_voiture_dep = nb_voiture_commune_dep2.groupby(['nom_departement', 'code_dep', 'dept_code_name', "annee"]).agg({
    'nb_vp_rechargeables_el': 'sum',
    'nb_vp': 'sum'
}).reset_index()

# Calcul du ratio électrique
nb_voiture_commune_dep2["ratio_elec_total"] = nb_voiture_commune_dep2["nb_vp_rechargeables_el"] / nb_voiture_commune_dep2["nb_vp"]

# Filtrage pour les années 2020 et 2021
nb_voiture_commune_dep = nb_voiture_commune_dep2[nb_voiture_commune_dep2["annee"].isin([2020, 2021])]
nb_voiture_commune_dep = nb_voiture_commune_dep.drop(columns=["annee"]).groupby(['nom_departement', 'code_dep']).agg({
    'nb_vp_rechargeables_el': 'sum',
    'nb_vp': 'sum'
}).reset_index()

# Export des résultats finaux
nb_voiture_commune_dep.to_csv("C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\Dossier_git\\data\\nb_voiture_commune.csv", index=False)
nb_voiture_dep.to_csv("C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\Dossier_git\\data\\nb_voiture_dep.csv", index=False)
nb_voiture_reg.to_csv("C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\Dossier_git\\data\\nb_voiture_reg.csv", index=False)


# Population

In [149]:
import pandas as pd

# Chargement des données
population = pd.read_csv("C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\Dossier_git\\data\\recensementNettoye.csv")
code_postal1 = pd.read_csv("C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\PIP\\DonneesBrutes\\Voiture\\code_postal.csv", sep=',')

# Conversion de la colonne 'reg' en chaîne de caractères
population["reg"] = population["reg"].astype(str)

# Création du DataFrame des départements
departement = code_postal1[["code_departement", "nom_departement"]].drop_duplicates()
departement["code_departement"] = departement["code_departement"].astype(str)
departement["code_departement"] = departement["code_departement"].apply(lambda x: x.zfill(2))  # Ajout de zéros à gauche pour uniformiser les codes

# Création du DataFrame des régions
region = code_postal1[["code_departement", "code_region", "nom_region"]].drop_duplicates()
region["code_departement"] = region["code_departement"].astype(str)
region["code_departement"] = region["code_departement"].apply(lambda x: x.zfill(2))

# Jointures des départements et des régions avec la population
population = population.merge(departement, left_on="dep", right_on="code_departement", how="left")
population = population.merge(region, left_on="dep", right_on="code_departement", how="left")

# Sélection des colonnes finales pour le fichier de sortie
colonnes_finales = [
    "objectid", "reg", "nom_region", "dep", "nom_departement", "cv", "codgeo_insee", 
    "libgeo", "p13_pop", "p14_pop", "p15_pop", "p16_pop", "p17_pop", 
    "p18_pop", "p19_pop", "p20_pop", "p21_pop", "p22_pop"
]
population = population[colonnes_finales]

# Export des données nettoyées
population.to_csv("C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\Dossier_git\\data\\population2.csv", index=False)


# Graphe traffic

In [ ]:
import pandas as pd
import json
from folium import Choropleth, LayerControl, GeoJsonTooltip

# Charger les données TMJA
chemin_tmja = "C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\PIP\\DonneesBrutes\\Voiture\\TMJA_2019.csv"
table = pd.read_csv(chemin_tmja)
table['TMJA'] = pd.to_numeric(table['TMJA'])  # Convertir la colonne TMJA en numérique
tmja_dep = table.groupby('depPrD')['TMJA'].mean()  # Calculer la moyenne TMJA par département

# Charger le fichier GeoJSON des départements français
chemin_geojson = "C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\PIP\\DonneesBrutes\\Voiture\\france_departments.geojson"
with open(chemin_geojson, 'r') as f:
    geo_departements = json.load(f)

# Préparer les données TMJA pour la fusion
tmja_dep_df = tmja_dep.reset_index()  # Convertir la série en DataFrame
tmja_dep_df.columns = ['code_departement', 'tmja_moyen']  # Renommer les colonnes
tmja_dep_df["code_departement"] = tmja_dep_df["code_departement"].astype(str).apply(lambda x: x.zfill(2))  # Uniformiser les codes

# Sauvegarder les données TMJA par département
chemin_tmja_dep_csv = "C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\Dossier_git\\data\\tmja_dep_df.csv"
tmja_dep_df.to_csv(chemin_tmja_dep_csv, index=False)

# Charger les informations des départements et des régions
code_postal1 = pd.read_csv(chemin_tmja)  # Remplacez par un fichier pertinent si ce n'est pas celui utilisé ici
departement = code_postal1[["code_departement", "nom_departement"]].drop_duplicates()
departement["code_departement"] = departement["code_departement"].astype(str).apply(lambda x: x.zfill(2))

region = code_postal1[["code_departement", "code_region", "nom_region"]].drop_duplicates()
region["code_departement"] = region["code_departement"].astype(str).apply(lambda x: x.zfill(2))

# Fusionner les données TMJA avec les données régionales
tmja_reg = tmja_dep_df.merge(region, on="code_departement", how="left")
tmja_reg = tmja_reg[["code_region", "tmja_moyen", "nom_region"]]

# Agréger les données TMJA par région
tmja_reg["code_region"] = tmja_reg["code_region"].astype(int).astype(str)
tmja_reg = tmja_reg.groupby(["code_region", "nom_region"]).agg({'tmja_moyen': 'mean'}).reset_index()

# Sauvegarder les données TMJA par région
chemin_tmja_reg_csv = "C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\Dossier_git\\data\\tmja_reg.csv"
tmja_reg.to_csv(chemin_tmja_reg_csv, index=False)

# Charger le fichier des données cartographiques pour la visualisation
chemin_carte_html = "C:\\Users\\MRH4\\OneDrive - MODEL RH\\Bureau\\PIP\\DonneesBrutes\\Voiture\\voitures-par-commune-par-energie.json"
with open(chemin_carte_html, "r", encoding="utf-8") as file:
    carte_html = file.read()
